In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_to_invasive = '/content/drive/MyDrive/aqua_data/invasive'
path_to_non_invasive = '/content/drive/MyDrive/aqua_data/non_invasive'


In [ ]:
import os
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

def load_images_and_labels(image_paths, label):
    images = []
    labels = []
    try:
        for image_path in os.listdir(image_paths):
            img = load_img(os.path.join(image_paths, image_path), target_size=(32, 32))
            img = img_to_array(img)
            images.append(img)
            labels.append(label)
    except Exception as e:
        print(f"Error: {e}")
    return np.array(images), np.array(labels)


x_invasive, y_invasive = load_images_and_labels(path_to_invasive, 1)
x_non_invasive, y_non_invasive = load_images_and_labels(path_to_non_invasive, 0)


Error: cannot identify image file <_io.BytesIO object at 0x7db278f01580>
Error: cannot identify image file <_io.BytesIO object at 0x7db1e42dd3f0>


In [ ]:
from sklearn.model_selection import train_test_split

x = np.concatenate((x_invasive, x_non_invasive), axis=0)
y = np.concatenate((y_invasive, y_non_invasive), axis=0)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255


In [ ]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


In [ ]:
print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")


x_train shape: (1397, 32, 32, 3) - y_train shape: (1397, 2)
x_test shape: (350, 32, 32, 3) - y_test shape: (350, 2)


In [ ]:
pip install tensorflow-addons


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.02),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(x_train)

In [ ]:
num_classes = 2
input_shape = (32, 32, 3)

In [ ]:
def create_encoder():
    resnet = keras.applications.ResNet50V2(
        include_top=False, weights=None, input_shape=input_shape, pooling="avg"
    )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model


encoder = create_encoder()
encoder.summary()

learning_rate = 0.001
batch_size = 265
hidden_units = 512
projection_units = 128
num_epochs = 50
dropout_rate = 0.5
temperature = 0.05

Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, 32, 32, 3)         7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23564807 (89.89 MB)
Trainable params: 23519360 (89.72 MB)
Non-trainable params: 45447 (177.53 KB)
_________________________________________________________________


In [ ]:
from keras.losses import CategoricalCrossentropy

def create_classifier(encoder, trainable=True):
    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=CategoricalCrossentropy(),  # Changed here
        metrics=['accuracy']  # Simplified metric for clarity
    )
    return model

# Create the encoder and classifier again
encoder = create_encoder()
classifier = create_classifier(encoder)

# Now fit the model
history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

# Evaluate the model
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")


Epoch 1/50
6/6 [==============================] - 103s 13s/step - loss: 1.5838 - accuracy: 0.6664
Epoch 2/50
6/6 [==============================] - 84s 14s/step - loss: 0.2855 - accuracy: 0.8912
Epoch 3/50
6/6 [==============================] - 82s 13s/step - loss: 0.2004 - accuracy: 0.9477
Epoch 4/50
6/6 [==============================] - 86s 14s/step - loss: 0.1601 - accuracy: 0.9578
Epoch 5/50
6/6 [==============================] - 84s 14s/step - loss: 0.0751 - accuracy: 0.9742
Epoch 6/50
6/6 [==============================] - 85s 14s/step - loss: 0.0700 - accuracy: 0.9764
Epoch 7/50
6/6 [==============================] - 83s 13s/step - loss: 0.0464 - accuracy: 0.9800
Epoch 8/50
6/6 [==============================] - 85s 14s/step - loss: 0.0497 - accuracy: 0.9800
Epoch 9/50
6/6 [==============================] - 83s 13s/step - loss: 0.0343 - accuracy: 0.9893
Epoch 10/50
6/6 [==============================] - 86s 14s/step - loss: 0.0426 - accuracy: 0.9807
Epoch 11/50
6/6 [===========

In [ ]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super().__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):
        # Normalize feature vectors
        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)
        # Compute logits
        logits = tf.divide(
            tf.matmul(
                feature_vectors_normalized, tf.transpose(feature_vectors_normalized)
            ),
            self.temperature,
        )
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="with_projection-head"
    )
    return model

In [ ]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()




Model: "with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functiona  (None, 2048)              23564807  
 l)                                                              
                                                                 
 dense_8 (Dense)             (None, 128)               262272    
                                                                 
Total params: 23827079 (90.89 MB)
Trainable params: 23781632 (90.72 MB)
Non-trainable params: 45447 (177.53 KB)
_________________________________________________________________


In [ ]:
history = encoder_with_projection_head.fit(
    x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs
)

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node softmax_cross_entropy_with_logits_1 defined at (most recent call last):
<stack traces unavailable>
logits and labels must be broadcastable: logits_size=[265,265] labels_size=[530,265]
	 [[{{node softmax_cross_entropy_with_logits_1}}]] [Op:__inference_train_function_176764]

In [ ]:
classifier = create_classifier(encoder, trainable=False)

history = classifier.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=num_epochs)

accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

In [ ]:
87

In [ ]:
"Test accuracy: 89.43%\n"